In [10]:
from __future__ import print_function

!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
import numpy as np
import pandas as pd
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

Requirement already up-to-date: google-api-python-client in /anaconda3/lib/python3.7/site-packages (1.7.8)
Requirement already up-to-date: google-auth-httplib2 in /anaconda3/lib/python3.7/site-packages (0.0.3)
Requirement already up-to-date: google-auth-oauthlib in /anaconda3/lib/python3.7/site-packages (0.3.0)


In [11]:
from urllib.parse import urlencode
from urllib.request import urlretrieve
from urllib.request import urlopen
from urllib.request import HTTPError


In [12]:
!pip install gmaps

In [13]:
with open('apikey.txt') as f:
    api_key = f.readline()
    f.close

In [14]:
import gmaps
gmaps.configure(api_key=api_key)

In [15]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
arj_ID = '18XjdaJE7ZGaRNOHTj5Fw3Z0kMLpvbyfrqEYF2IvB390'

arj_name = 'Sheet1!A2:A'

riley_ID = '1MZIa4_FEcoXk41eZWmiW8xgrPzimGEYlhhn3iXB9rf4'

def sheet_to_list(ID, name):
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=ID,
                                range=name).execute()
    values = result.get('values', [])

    colleges = [loc[0] for loc in values if len(loc) != 0]
    return colleges

arj_colleges = sheet_to_list(arj_ID, arj_name)
arj_ranks = sheet_to_list(arj_ID, 'Sheet1!C2:C')
riley_colleges = sheet_to_list(riley_ID, 'Sheet1!A2:A')
riley_locs = sheet_to_list(riley_ID, 'Sheet1!B2:B')

In [16]:
riley_cities = [string.split(', ')[0] for string in riley_locs]
riley_states = [string.split(', ')[1] for string in riley_locs]

cities, states = [], []
lat, lng = [], []

for i in range(len(riley_cities)):
    city, state = riley_cities[i], riley_states[i]
    corr_city = cities_data['city']==city
    corr_state = cities_data['state_id']==state
    corr_df = cities_data[corr_city & corr_state]
    try:
        lat += [corr_df['lat'].values[0]]
        lng += [corr_df['lng'].values[0]]
    except IndexError:
        print(f'idk where {city} is :(')
    cities += [city]
    states += [state]

riley_cities = pd.DataFrame({'City':cities, 'State':states, 'Latitude':lat, 'Longitude':lng })

NameError: name 'cities_data' is not defined

In [ ]:
cities_data = pd.read_csv('uscitiesv1.4.csv') #instead of doing this, query google!

In [ ]:
#have to use a package.

def query_college(college):
    try: 
        query =f'''https://en.wikipedia.org/wiki/{college}'''
        urlpath = urlopen(query)
    except HTTPError:
        college = college.replace(" ", "_")
        query =f'''https://en.wikipedia.org/wiki/{college}'''
        urlpath = urlopen(query)
    string = urlpath.read().decode('utf-8')
    city_index = string.split('>').index('Location</th') + 4
    if ', ' not in string.split('>')[city_index]:
        city = string.split('>')[city_index][:-3]
        state_index = string.split('>').index('Location</th') + 8
        state = string.split('>')[state_index][:-3]
    else:
        both = string.split('>')[city_index].split(', ')
        city = both[0]
        state = both[1][:-3]
    try:
        ID = state_to_ID(state)
    except ValueError:
        try:
            state_index = string.split('>').index('Location</th') + 8
            state = string.split('>')[state_index][:-3]
            ID = state_to_ID(state)
        except ValueError:
            state_index = string.split('>').index('Location</th') + 7
            state = string.split('>')[state_index][:-5]
            ID = state_to_ID(state)
    return city, ID


In [ ]:
#now query wikipedia for the state abbreviations.
def state_to_ID(state, verbose=False):
    query = 'https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations'
    urlpath = urlopen(query)
    string = urlpath.read().decode('utf-8')
    ID_index = string.split('>').index(state + '</a') + 12
    ID = string.split('>')[ID_index][:-6]
    return ID

In [ ]:
lat, lng = [], []
cities, states = [], []
ranks = []
for i in range(len(arj_colleges)):
    city, state = query_college(arj_colleges[i])
    corr_city = cities_data['city']==city
    corr_state = cities_data['state_id']==state
    corr_df = cities_data[corr_city & corr_state]
    try:
        lat += [corr_df['lat'].values[0]]
        lng += [corr_df['lng'].values[0]]
        cities += [city]
        states += [state]
        ranks += [arj_ranks[i]]
    except IndexError:
        print(f'idk where {city} is :(')
    

arj_cities = pd.DataFrame({'City':cities, 'State':states, 'Latitude':lat, 'Longitude':lng, 'Ranks':ranks })

arj1 = arj_cities[arj_cities['Ranks'] == '1']
arj2 = arj_cities[arj_cities['Ranks'] == '2']
arj3 = arj_cities[arj_cities['Ranks'] == '3']

plot_arj_cities1 = arj1[['Latitude', 'Longitude']]
plot_arj_cities2 = arj2[['Latitude', 'Longitude']]
plot_arj_cities3 = arj3[['Latitude', 'Longitude']]

plot_riley_cities = riley_cities[['Latitude', 'Longitude']]

arjun1_layer = gmaps.symbol_layer(
    plot_arj_cities1, fill_color='green', stroke_color='green', scale=3)
arjun2_layer = gmaps.symbol_layer(
    plot_arj_cities2, fill_color='yellow', stroke_color='yellow', scale=3)
arjun3_layer = gmaps.symbol_layer(
    plot_arj_cities3, fill_color='red', stroke_color='red', scale=3)
riley_layer = gmaps.symbol_layer(
    plot_riley_cities, fill_color='purple', stroke_color='purple', scale=2)
fig = gmaps.figure()
fig.add_layer(arjun1_layer)
fig.add_layer(arjun2_layer)
fig.add_layer(arjun3_layer)
fig.add_layer(riley_layer)
fig

In [ ]:
# weights = earthquake_df['magnitude']
# #Set up your map
# fig = gmaps.figure()
# fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
# fig

In [ ]:
arjun1_layer = gmaps.symbol_layer(
    plot_arj_cities1, fill_color='green', stroke_color='green', scale=2)
arjun2_layer = gmaps.symbol_layer(
    plot_arj_cities2, fill_color='yellow', stroke_color='yellow', scale=2)
arjun3_layer = gmaps.symbol_layer(
    plot_arj_cities3, fill_color='red', stroke_color='red', scale=2)
riley_layer = gmaps.symbol_layer(
    plot_riley_cities, fill_color='purple', stroke_color='purple', scale=3)
fig = gmaps.figure()
fig.add_layer(riley_layer)
fig.add_layer(arjun3_layer)
fig.add_layer(arjun2_layer)
fig.add_layer(arjun1_layer)
fig